<a href="https://colab.research.google.com/github/hera1228/DAP_2023/blob/main/dap_w11_pub_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. 가설 정하기**

통계 검정에서 가설은 두 가지로 분류됩니다: <br>
1. 귀무가설(Null Hypothesis, H0)
2. 대립가설(Alternative Hypothesis, H1 혹은 HA)

이 두 가설 중, 분석자가 사실인지를 확인하고자 하는 가설은 대립가설이며, 귀무가설이 사실인지 아닌지를 확인함으로써 대립가설이 사실인지를 판단하는 것입니다.<br><br>

예를 들어, 흡연 여부와 폐암의 관련성이 있는지를 확인하고자 하는 경우를 봅니다.<br><br>

일반적으로 세간에는 담배를 피는 것이 폐암 발병 여부와 관련이 없다고 생각한다고 가정합시다.<br>
그런데, 폐암에 걸리는 이유를 연구하는 연구자 A가 흡연과 폐암에는 직접적인 연관성이 있을 것 같다는 생각을 하여 통계분석을 통해 이의 진위여부를 확인하려고 합니다. <br>
이 때, 연구자 A의 입장에서 확인하고자 하는 것은 **"흡연 여부와 폐암 발생 여부에는 관련성이 *있다*"** 이며, 이는 일반적으로 알려진 것인 **"흡연 여부와 폐암 발생 여부에는 관련성이 *없다*"**라는 가설이 거짓임을 확인함으로써 받아들일 수 있는 가설이 됩니다. <br>
이 경우, **"흡연 여부와 폐암 발생 여부에는 관련성이 *있다*"** 는 **대립가설** 이 되고, **"흡연 여부와 폐암 발생 여부에는 관련성이 *없다*"** 는 **귀무가설** 이 됩니다.

In [ ]:
# 1-예시. 흡연과 폐암

import pandas as pd
from scipy.stats import ttest_ind
from sklearn.utils import resample

df_lung = pd.read_csv("https://raw.githubusercontent.com/6n23rp/ZGFwMjNtaWRkYXRh/main/c3VydmV5IGx1bmcgY2FuY2Vy.csv")
df_lung.replace({
    1 : False,
    2 : True,
    'NO' : 0,
    'YES' : 1
}, inplace = True)

print(df_lung.head(1))

# 독립표본 T-분석(Independent Two-Sample t-test)을 통해 가설을 검사한다고 합니다.
# nonsmokers는 무작위로 추출된 100명의 비흡연자 표본입니다.
nonsmokers = resample(df_lung[~df_lung['SMOKING']], n_samples = 100, random_state = 42)
# smokers는 무작위로 추출된 100명의 흡연자 표본입니다.
smokers = resample(df_lung[df_lung['SMOKING']], n_samples = 100, random_state = 24)

# 각 표본 DataFrame의 'LUNG_CANCER' 열은 폐암을 가진 경우 1, 폐암을 가지지 않은 경우 0을 가집니다.
# 두 표본 집단의 폐암 여부 평균이 크게 다르다면 흡연 여부와 폐암 발병 여부는 서로 관련이 없다는 귀무가설(H0)을 기각하여,
# 흡연 여부와 폐암 발병 여부에는 관련이 있다는 대립가설(H1)을 받아들일 수 있게 됩니다.

# ttest_ind는 두 표본 집단의 평균값을 비교하여 p값을 반환합니다. 평균값의 차이가 작을수록 p값이 크고(최대 1), 차이가 클수록 p값이 작습니다(최소 0).
_, p_value = ttest_ind(nonsmokers['LUNG_CANCER'], smokers['LUNG_CANCER'])

# 유의수준이 0.05라고 가정합니다.
a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각하여 대립가설을 받아들입니다. p값이 유의수준보다 크거나 같은 경우 귀무가설을 받아들이고 대립가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 실행 결과를 보면 p값이 0.63..으로 유의수준보다 크기 때문에 귀무가설을 받아들이고, 흡연 여부와 폐암 발병 여부는 관련이 있다는 대립가설을 기각합니다.
# 따라서, 이 분석의 결론은 "흡연 여부와 폐암 발병 여부는 관련이 없다" 입니다.

  GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0      M   69    False            True     True          False   

   CHRONIC DISEASE  FATIGUE   ALLERGY   WHEEZING  ALCOHOL CONSUMING  COUGHING  \
0            False      True     False      True               True      True   

   SHORTNESS OF BREATH  SWALLOWING DIFFICULTY  CHEST PAIN  LUNG_CANCER  
0                 True                   True        True            1  
P-value [0.6393698855947099] >= Significant level [0.05]; H0 is accepted.


## **2. 유의수준 정하기**

1.의 예시를 보면, 두 표본간의 평균값이 별 차이가 나지 않는 경우 귀무가설을 받아들이고, 대립가설을 기각하였습니다. 이 때 판단 기준인 "평균값의 차이" 정도를 결정하는 것이 유의수준입니다.

In [ ]:
from scipy.stats import ttest_ind

# 아래와 같은 두 표본이 있습니다.
smokers =     [1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
nonsmokers =  [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]

# ttest_ind는 두 표본 집단의 평균값을 비교하여 p값을 반환합니다.
_, p_value = ttest_ind(nonsmokers, smokers)

# 유의수준을 0.05으로 지정합니다.
a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각하여 대립가설을 받아들입니다. p값이 유의수준보다 크거나 같은 경우 귀무가설을 받아들이고 대립가설을 기각합니다.
def fun(p_value, a):
  if p_value < a:
    print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
  else:
    print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 유의수준이 0.05인 경우, p값이 0.05보다 크기 때문에 귀무가설을 받아들이고, 대립가설을 기각합니다.
fun(p_value, a)

# 유의수준이 0.1이라고 합니다.
a = 0.1

# 유의수준이 0.1인 경우에는 p값이 0.1보다 작기 때문에 귀무가설을 기각하고, 대립가설을 받아들입니다.
fun(p_value, a)

# 즉, 유의수준이 클수록 대립가설을 받아들이는 기준이 유하다는 것을 확인할 수 있습니다.

P-value [0.08055387210850923] >= Significant level [0.05]; H0 is accepted.
P-value [0.08055387210850923] < Significant level [0.1]; H1 is accepted.


## **3. 단일표본 T-분석 (One-Sample t-test)**

단일표본 T-분석은 집단의 어떠한 변수의 평균이 어떠한 값과 같거나 비슷할지를 판단하고자 할 때 사용합니다.<br><br>

예를 들어, 타이타닉호 승객의 평균 나이는 30세 혹은 30세 정도일 것인지를 확인하고자 합니다. 이 때 귀무가설은 "타이타닉호 승객의 평균 나이는 30세이다"이고, 대립가설은 "타이타닉호 승객의 평균 나이는 30세가 아니다"가 됩니다. 아래 코드 셀에 이 가설을 확인하는 단일표본 T-분석의 예시가 있습니다.

In [ ]:
# 3-예시.

# 단일표본 T-분석에는 scipy의 ttest_1samp 메서드를 이용합니다.
from scipy.stats import ttest_1samp

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 승객들의 나이 정보를 이용합니다. (결측값은 제거합니다.)
ages = df_titanic["Age"].dropna()

# 평균을 계산할 집단의 변수인 ages와 비교 대상 값인 30을 ttest_1samp의 파라미터로 전달하여 p값을 계산합니다.
_, p_value = ttest_1samp(ages, 30) #파라미터(리스트, 임의의 평균값)

# 유의수준을 0.01로 선택합니다.
a = 0.01

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 결과를 보면, p값이 유의수준보다 크기 때문에 귀무가설을 받아들입니다.
# 따라서, 이 분석의 결론은 "타이타닉호 승객의 평균 나이는 30세이다"가 됩니다.

# 실제로 평균 나이를 확인해보면 29.69...으로 30세에 가까운 것을 확인할 수 있습니다.
print(ages.mean())

P-value [0.5801231230388639] >= Significant level [0.01]; H0 is accepted.
29.69911764705882


In [ ]:
df_titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 3. df_titanic을 이용하여, 임의의 귀무가설과 대립가설을 수립해 단일표본 T-분석을 수행하세요.
# 유의수준은 임의로 설정하세요.
# 출력은 실제 평균값 확인을 제외하고 예시와 같은 방식으로 하세요.
from scipy.stats import ttest_1samp

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 아래에 귀무가설(H0)과 대립가설(H1)을 기재하세요.
# H0
"""
HO="생존한 사람의 pclass는 비슷하다"
"""

# H1
"""
H1="생존한 사람의 pclass는 비슷하지 않다"
"""

# 아래 구간 내에서 T-분석을 수행하세요.
######>
pclass= df_titanic[df_titanic["Survived"]==1]
pclass = df_titanic["Pclass"].dropna()

_, p_value = ttest_1samp(pclass, 2)
a = 0.01

if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

######<

# 아래에 분석의 결론을 기재하세요.
"""
p-value값이 굉장히 낮게 나왔으므로 대립가설이 선택된다.
따라서 생존한 사람의 pclass는 비슷하지 않다.
"""

P-value [1.4702052068990055e-26] < Significant level [0.01]; H1 is accepted.


'\np-value값이 굉장히 낮게 나왔으므로 대립가설이 선택된다.\n따라서 생존한 사람의 pclass는 비슷하지 않다.\n'

## **4. 두 집단간 평균 비교 -- 독립표본 T-분석 (Independent Two-Sample t-test)**

독립표본 T-분석은 서로 독립인 두 표본의 평균이 비슷한지를 판단할 때 사용합니다.<br><br>

예를 들어, 타이타닉호의 승객 중, 생존한(Survived = 1) 승객들과 사망한(Survived = 0) 승객들의 평균 운임(Fare)이 같거나 비슷한지를 확인하고자 합니다. 이 때, 귀무가설은 "생존한 승객과 사망한 승객의 평균 운임은 같다"이고, 대립가설은 "생존한 승객과 사망한 승객의 평균 운임은 같지 않다"가 됩니다.

In [ ]:
# 4-예시.

# 독립표본 T-분석에는 scipy의 ttest_ind 메서드를 이용합니다.
from scipy.stats import ttest_ind

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 생존 여부 혹은 운임 정보가 없는 레코드는 제거합니다.
df_titanic = df_titanic.dropna(subset = ["Survived", "Fare"])

# 생존자들의 운임 정보를 추출합니다.
survived_fares = df_titanic[df_titanic["Survived"] == 1]["Fare"]

# 사망자들의 운임 정보를 추출합니다.
dead_fares = df_titanic[df_titanic["Survived"] == 0]["Fare"]

# 평균을 비교할 두 표본을 ttest_ind의 파라미터로 전달하여 p값을 계산합니다.
_, p_value = ttest_ind(survived_fares, dead_fares) #-: ..?버린거..?

# 유의수준을 0.05로 선택합니다.
a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 결과를 보면, p값이 유의수준보다 작기 때문에 대립가설을 받아들입니다.
# 따라서, 이 분석의 결론은 "생존한 승객과 사망한 승객의 평균 운임은 같지 않다"가 됩니다.

# 실제로 두 표본의 평균을 확인해보면, 약 48과 22로 상당히 큰 차이가 있음을 확인할 수 있습니다.
print(survived_fares.mean())
print(dead_fares.mean())

P-value [6.120189341924198e-15] < Significant level [0.05]; H1 is accepted.
48.39540760233918
22.117886885245902


In [ ]:
# 4. df_titanic을 이용하여, 임의의 귀무가설과 대립가설을 수립해 독립표본 T-분석을 수행하세요.
# 유의수준은 임의로 설정하세요.
# 출력은 실제 평균 비교를 제외하고 예시와 같은 방식으로 하세요.
from scipy.stats import ttest_ind

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 아래에 귀무가설(H0)과 대립가설(H1)을 기재하세요.
# H0
"""
H0 = " 사망자와 생존자의 나이는 비슷하다. "
"""

# H1
"""
H1 = " 사망자와 생존자의 나이는 비슷하지 않다. "
"""

# 아래 구간 내에서 T-분석을 수행하세요.
######>
df_titanic = df_titanic.dropna(subset = ["Survived", "Age"])

survived_age = df_titanic[df_titanic["Survived"] == 1]["Age"]

# 사망자들의 운임 정보를 추출합니다.
dead_age = df_titanic[df_titanic["Survived"] == 0]["Age"]

# 평균을 비교할 두 표본을 ttest_ind의 파라미터로 전달하여 p값을 계산합니다.
_, p_value = ttest_ind(survived_age, dead_age)

a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

######<

# 아래에 분석의 결론을 기재하세요.
"""
 p_value 가 매우 작은 값으로 a(유의수준)값이 더 크므로 대립가설을 선택한다.
 따라서 사망자와 생존자의 나이는 비슷하지 않다.
"""

P-value [0.03912465401348249] < Significant level [0.05]; H1 is accepted.


'\n p_value 가 매우 작은 값으로 a(유의수준)값이 더 크므로 대립가설을 선택한다.\n 따라서 사망자와 생존자의 나이는 비슷하지 않다.\n'

In [ ]:
survived_age

1      38.0
2      26.0
3      35.0
8      27.0
9      14.0
       ... 
875    15.0
879    56.0
880    25.0
887    19.0
889    26.0
Name: Age, Length: 290, dtype: float64

## **5. 대응표본 T-분석 (Paired Sample t-test)**

대응표본 T-분석은 동일 표본에 발생한 특정 사건 전과 후로 구분된 두 관측점 사이 표본 내 각 개체별 변화량의 평균이 유의미한지를 확인할 때 사용합니다.

집단 전체의 평균의 차이가 아니라, 집단 내 객체들의 사건 전후 차이의 평균을 이용합니다.<br><br>

예시로는 다이어트 약품 투약 전과 후의 체중 비교, 어느 특강을 수강하기 전과 후의 학업성취도 비교 등이 있겠습니다. <br><br>

현실 데이터셋을 구하기 쉽지 않기 때문에, 임의로 생성된 데이터를 이용하는 예시 코드를 제시합니다.

In [ ]:
# 5-예시.

import numpy as np
import math

# 대응표본 T-분석에는 ttest_rel 메서드를 이용합니다.
from scipy.stats import ttest_rel

# 10대의 각기 다른 성능을 가진 PC가 동일한 프로그램을 실행하였을 때, 실행 완료까지 걸리는 시간을 보유하고 있다고 합니다.
befores = np.array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

# 이 프로그램에 어떠한 패치가 적용된 이후, 각 PC의 실행 완료 시간을 측정하였다고 합니다.
np.random.seed(42)
afters = befores + np.random.normal(0, 5, size = len(befores))

# 이 패치가 실행속도에 영향을 주었는지를 검사하고자 합니다.
# 귀무가설(H0)은 "패치는 프로그램의 실행 속도에 영향을 주지 않았다"입니다.
# 대립가설(H1)은 "패치는 프로그램의 실행 속도에 영향을 주었다"입니다.

# 두 시점의 관측치들을 ttest_rel의 파라미터로 넘겨 p값을 계산합니다.
_, p_value = ttest_rel(befores, afters)

# 유의수준을 0.05로 선택합니다.
a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 결과를 보면, p값이 유의수준보다 크기 때문에 귀무가설을 받아들입니다.
# 따라서, 이 분석의 결론은 "패치는 프로그램의 실행 속도에 영향을 주지 않았다"가 됩니다.

P-value [0.08168120130914497] >= Significant level [0.05]; H0 is accepted.


## **6. 카이제곱 검정 (Chi-squared test)**

카이제곱 검정은 어떠한 집단 내 범주형 변수별 빈도수의 기대치와 실제 관측치의 비교를 수행할 때 사용합니다.

예를 들어, 타이타닉 데이터셋의 생존 여부별 빈도 분포가 성별에 무관하게 동일한지를 확인하고자 합니다. 이 때, 귀무가설은 "성별에 따른 생존 여부 비율은 동일하다" 이고, 대립가설은 "성별에 따른 생존 여부 비율은 동일하지 않다"가 됩니다. 구체적으로, 여성의 생존자 비율을 구하고, 이 비율을 이용해 남성의 기대 생존자 빈도를 계산한 후, 남성의 실제 생존자 빈도를 추출해 실제 관측값과 기대값을 비교합니다.

In [ ]:
# 6-예시.

# 카이제곱 검정에는 scipy의 chisquare 메서드를 이용합니다.
from scipy.stats import chisquare

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 생존 여부 혹은 성별 정보가 없는 레코드는 제거합니다.
df_titanic = df_titanic.dropna(subset = ["Survived", "Sex"])

# 여성 승객의 생존자 비율을 계산합니다.
female_survival_perc = np.array(df_titanic[df_titanic["Sex"] == "female"]["Survived"].value_counts().sort_index().values
                        / len(df_titanic[df_titanic["Sex"] == "female"]))

female_survival_perc

# 여성 승객의 생존자 비율을 이용해 남성 승객의 생존 여부별 예상 빈도수를 계산합니다.
male_survival_expectations = np.array(female_survival_perc) * len(df_titanic[df_titanic["Sex"] == "male"])

# 실제 남성 승객의 생존 여부별 빈도수를 추출합니다.
male_survival_observations = np.array(df_titanic[df_titanic["Sex"] == "male"]["Survived"].value_counts().sort_index().values)

# 여기서, 기대값은 male_survival_expectations가 됩니다.
# 관측값은 male_survival_observations가 됩니다.

# 또한, 귀무가설은 "성별에 따른 생존 여부 비율은 동일하다"이고, 대립가설은 "성별에 따른 생존 여부 비율은 동일하지 않다"가 됩니다.

# 관측값과 기대값을 chisquare의 파라미터로 전달하여 p값을 계산합니다.
_, p_value = chisquare(male_survival_observations, male_survival_expectations)

# 유의수준은 0.05로 선택합니다.
a = 0.05

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")

# 결과를 보면, p값이 유의수준보다 작기 때문에 대립가설을 받아들입니다.
# 따라서, 이 분석의 결론은 "성별에 따른 생존 여부 비율은 동일하지 않다"가 됩니다.

P-value [1.4288364090003192e-202] < Significant level [0.05]; H1 is accepted.


In [ ]:
female_survival_perc
male_survival_observations

array([468, 109])

In [ ]:
df_titanic

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
885,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39.0,0,5,382652,29.1250,NaN,Q
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
# 6. df_titanic을 이용하여, 임의의 귀무가설과 대립가설을 수립해 카이제곱 검정을 수행하세요.
# 유의수준은 임의로 설정하세요.
# 출력은 예시와 같은 방식으로 하세요.
from scipy.stats import chisquare

df_titanic = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# 아래에 귀무가설(H0)과 대립가설(H1)을 기재하세요.
# H0
"""
H0 : pcalss1에 탄 사람과 pclass3에 탄 사람의 생존비율은 같다.
"""

# H1
"""
H1 : pcalss1에 탄 사람과 pclass3에 탄 사람의 생존비율은 같지 않다.
"""


# 아래 구간 내에서 카이제곱 검정을 수행하세요.
######>
df_titanic = df_titanic.dropna(subset = ["Survived", "Pclass"])

pclass1_survival_perc = np.array(df_titanic[df_titanic["Pclass"] == 1]["Survived"].value_counts().sort_index().values
                        / len(df_titanic[df_titanic["Pclass"] == 1]))
pclass3_survival_expectations = np.array(pclass1_survival_perc) * len(df_titanic[df_titanic["Pclass"] == 3])

pclass3_survival_observations = np.array(df_titanic[df_titanic["Pclass"] == 3]["Survived"].value_counts().sort_index().values)

_, p_value = chisquare(pclass3_survival_observations, pclass3_survival_expectations)

# 유의수준은 0.01로 선택합니다.
a = 0.01

# p값이 유의수준보다 작은 경우 귀무가설을 기각합니다.
if p_value < a:
  print(f"P-value [{p_value}] < Significant level [{a}]; H1 is accepted.")
else:
  print(f"P-value [{p_value}] >= Significant level [{a}]; H0 is accepted.")
######<

# 아래에 분석의 결론을 기재하세요.
"""
p_value 가 매우 작은 값으로 a(유의수준)값이 더 크므로 대립가설을 선택한다.
 따라서 pclass1에 탄 사람과 pcalss3에 탄 사람의 생존비율은 다르다..
"""

P-value [1.2040677424018695e-70] < Significant level [0.01]; H1 is accepted.


'\np_value 가 매우 작은 값으로 a(유의수준)값이 더 크므로 대립가설을 선택한다.\n 따라서 pclass1에 탄 사람과 pcalss3에 탄 사람의 생존비율은 다르다..\n'